In [10]:
import numpy as np
import pandas as pd
import scipy.stats as stats


# Simulation parameters
n_simulations = 10**4  # Number of simulations to approximate the distribution
beta = 10**8  # Inverse temperature
lambda_step = 10**(-4)  # Step size for SGLD
gamma = 10**(-8)  # Regularization parameter
theta = 0  # Initial guess for theta

# Set quantile levels and number of iterations for SGLD
quantile_levels = [0.95, 0.99]
n_iterations = 10**6

# Define the parameters for normal distribution as per Table 1
normal_params = [
    {'mu': 0, 'sigma': 1},
    {'mu': 1, 'sigma': 2},
    {'mu': 3, 'sigma': 5}
]

# Define the degrees of freedom for Student's t-distribution as per Table 2
student_t_dfs = [10, 7, 3]


In [7]:
# Placeholder for the loss function V(theta)
def V(theta, returns, q, gamma):
    loss = np.mean([theta + (1/(1 - q)) * max(return_ - theta, 0) for return_ in returns]) + gamma * theta**2
    return loss

# Simulate asset returns
def simulate_asset_returns(dist, params, size):
    if dist == 'normal':
        return np.random.normal(params['mu'], params['sigma'], size)
    elif dist == 'student_t':
        return np.random.standard_t(params['df'], size)

# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return var, cvar



# SGLD Iteration for a single step
def sgld_step(theta, x, q, gamma, lambda_step):
    indicator_geq = 1 if x >= theta else 0
    indicator_less = 1 if x < theta else 0
    H_theta_x =  - q / (1 - q) + indicator_less / (1 - q) + 2 * gamma * theta
    theta = theta - lambda_step * H_theta_x + np.sqrt(2 * lambda_step / beta) * np.random.randn()
    return theta

# Run SGLD simulation for VaR and CVaR calculation
def run_sgld_simulation(dist, params, q, lambda_step, gamma, n_iterations):
    theta = 0  # Initial guess for theta
    for _ in range(n_iterations):
        x = simulate_asset_returns(dist, params, 1)
        theta = sgld_step(theta, x, q, gamma, lambda_step)
    return theta

# Adjusted get_results_for_quantile function
def get_results_for_quantile(dist_params, q, dist_type):
    results = []
    for params in dist_params:
        # Simulate returns and calculate VaR and CVaR
        if dist_type == 'normal':
            returns = simulate_asset_returns(dist_type, params, n_simulations)
        elif dist_type == 'student_t':
            # Adjusted for Student's t-distribution
            returns = np.random.standard_t(params, n_simulations)
        
        VaR_empirical, CVaR_empirical = calculate_var_cvar(returns, q)
        
        # Optimize theta using SGLD and calculate VaR and CVaR
        if dist_type == 'normal':
            theta = run_sgld_simulation(dist_type, params, q, lambda_step, gamma, n_iterations)
        elif dist_type == 'student_t':
            # Adjusted for Student's t-distribution
            theta = run_sgld_simulation(dist_type, {'df': params}, q, lambda_step, gamma, n_iterations)
        
        VaR_SGLD = theta
        CVaR_SGLD = V(theta, returns, q, gamma)  # This would typically use theta to calculate VaR_SGLD, CVaR_SGLD
        
        results.append([VaR_empirical, CVaR_empirical, VaR_SGLD, CVaR_SGLD])
        
    return results

# Collecting and organizing results into a DataFrame for each quantile level
results_data_1 = {
    '0.95': {
        'Normal': get_results_for_quantile(normal_params, 0.95, 'normal'),
    },
    '0.99': {
        'Normal': get_results_for_quantile(normal_params, 0.99, 'normal'),
    }
}

results_data_2 = {
    '0.95': {
        'Student\'s t': get_results_for_quantile(student_t_dfs, 0.95, 'student_t')
    },
    '0.99': {
        'Student\'s t': get_results_for_quantile(student_t_dfs, 0.99, 'student_t')
    }
}
# Creating a DataFrame from the results_data
results_df_1 = pd.DataFrame(
    index=pd.MultiIndex.from_product([['Normal'], ['μ=0, σ=1', 'μ=1, σ=2', 'μ=3, σ=5']], 
                                     names=['Distribution', 'Parameters']),
    columns=pd.MultiIndex.from_product([['0.95', '0.99'], ['VaR', 'CVaR', 'VaR_SGLD', 'CVaR_SGLD']], 
                                       names=['Quantile', 'Metric'])
)

results_df_2 = pd.DataFrame(
    index=pd.MultiIndex.from_product([[ 'Student\'s t'], ['d.f.=10', 'd.f.=7', 'd.f.=3']], 
                                     names=['Distribution', 'Parameters']),
    columns=pd.MultiIndex.from_product([['0.95', '0.99'], ['VaR', 'CVaR', 'VaR_SGLD', 'CVaR_SGLD']], 
                                       names=['Quantile', 'Metric'])
)

# Filling the DataFrame with results
for q in ['0.95', '0.99']:
    for dist_type in ['Normal']:
        if dist_type == 'Normal':
            params_list = ['μ=0, σ=1', 'μ=1, σ=2', 'μ=3, σ=5']
        for idx, params in enumerate(params_list):
            results_df_1.loc[(dist_type, params), (q, 'VaR')] = results_data_1[q][dist_type][idx][0]
            results_df_1.loc[(dist_type, params), (q, 'CVaR')] = results_data_1[q][dist_type][idx][1]
            results_df_1.loc[(dist_type, params), (q, 'VaR_SGLD')] = results_data_1[q][dist_type][idx][2]
            results_df_1.loc[(dist_type, params), (q, 'CVaR_SGLD')] = results_data_1[q][dist_type][idx][3]
            

# Filling the DataFrame with results
for q in ['0.95', '0.99']:
    for dist_type in ['Student\'s t']:
        if dist_type == 'Student\'s t':
            params_list = ['d.f.=10', 'd.f.=7', 'd.f.=3']
        for idx, params in enumerate(params_list):
            results_df_2.loc[(dist_type, params), (q, 'VaR')] = results_data_2[q][dist_type][idx][0]
            results_df_2.loc[(dist_type, params), (q, 'CVaR')] = results_data_2[q][dist_type][idx][1]
            results_df_2.loc[(dist_type, params), (q, 'VaR_SGLD')] = results_data_2[q][dist_type][idx][2]
            results_df_2.loc[(dist_type, params), (q, 'CVaR_SGLD')] = results_data_2[q][dist_type][idx][3]


# Print the table
print("Table 1 & 2: VaR and CVaR for Normal and Student's t distributions")

print(results_df_1)

print(results_df_2)




Table 1 & 2: VaR and CVaR for Normal and Student's t distributions
Quantile                      0.95                                       0.99  \
Metric                         VaR       CVaR  VaR_SGLD  CVaR_SGLD        VaR   
Distribution Parameters                                                         
Normal       μ=0, σ=1     1.644405   2.070598   1.68262   2.071605   2.383647   
             μ=1, σ=2     4.253336   5.041455  4.288314   5.042204     5.5228   
             μ=3, σ=5    11.142476  13.184636  11.27332  13.188481  14.686339   

Quantile                                                  
Metric                        CVaR   VaR_SGLD  CVaR_SGLD  
Distribution Parameters                                   
Normal       μ=0, σ=1     2.699017    2.30929   2.705473  
             μ=1, σ=2     6.202443   5.789447   6.239547  
             μ=3, σ=5    16.393278  14.680923  16.393331  
Quantile                     0.95                                   0.99  \
Metric          

OSError: Cannot save file into a non-existent directory: 'T1'